In [20]:
'''%pip install --disable-pip-version-check \
        torch==1.13.1 \
        torchdata==0.5.1 --quiet

%pip install \
        transformers==4.27.2 \
        datasets==2.11.0 --quiet'''

'%pip install --disable-pip-version-check         torch==1.13.1         torchdata==0.5.1 --quiet\n\n%pip install         transformers==4.27.2         datasets==2.11.0 --quiet'

In [21]:
'''%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet'''

'%pip install --upgrade pip\n%pip install --disable-pip-version-check     torch==1.13.1     torchdata==0.5.1 --quiet\n\n%pip install     transformers==4.27.2     datasets==2.11.0  --quiet'

In [3]:
from config import reddit_client_id, reddit_client_secret, reddit_user_agent

In [4]:
import praw
# Initialize the Reddit API wrapper
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Replace 'python' with the name of the subreddit you want to pull posts from
subreddit = reddit.subreddit('CryptoCurrency')

for submission in reddit.subreddit("CryptoCurrency").hot(limit=10):
    print(submission.title)

#TODO: extract headings and top level comments for top n threads (hot n?)
#TODO: store this where? Patch as json to some store - blob?

AMA Announcement: Unveiling the Future of Decentralized AI and Software Development with Tau Net ($AGRS) - March 25th, 6pm UTC - 1500 USDT Giveaway
Tutorial: Bridging MOON to Arbitrum One via Celer (Celer Bridge is now live!)
Ethereum Could Top $14,000 Next Year Alongside Bitcoin Boom: Standard Chartered
I analyzed the Last 3 Bitcoin halvings here's what I think will happen after the 2024 halving
Cardano Gets First Fiat-Backed Stablecoin with USDM
Daily Crypto Discussion - March 25, 2024 (GMT+0)
This Week in Crypto Games: 'Crypto Valleys' Blasts Off, Immutable and Polygon Plot $100M Fund
Litecoin is breaking out of 11Y old LTC/BTC ratio downtrend, next 1W candle on 25th March 2024
Professional Shitposting
Skybridge Capital Founder Advises 'Act Like You're Dead With Your Bitcoin and Don't Sell' 


In [ ]:
'''%pip install emoji'''

In [213]:
# compile the dataset
# convert emojis to plain text before writing to csv
from emoji import demojize
import re


submissions_dataset = []
comments_dataset = []

for submission in reddit.subreddit("CryptoCurrency").top('day',limit=10):
    submissions_dataset.append([submission.id, submission.title, submission.created, submission.score])
    for top_level_comment in submission.comments:
            comments_dataset.append([top_level_comment.id, re.sub(r'\s+', ' ', demojize(top_level_comment.body)), top_level_comment.created_utc, top_level_comment.score, submission.id])

C:\Users\olgakh\AppData\Local\Temp\ipykernel_14216\1999565252.py:10: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for submission in reddit.subreddit("CryptoCurrency").top('day',limit=10):


In [214]:
import csv

submissions_filename = "reddit_submissions_data.csv"
comments_filename = "reddit_comments_data.csv"

'''
# Define the fields and rows
# Do this the first time writing to the files

submission_fields = ['ID', 'Title', 'Unixtime', 'Upvotes']
comment_fields = ['ID','Comment', 'Unixtime', 'Upvotes', 'SubmissionID']
'''


with open(submissions_filename, 'a') as csvfile:
    # Create a writer object
    csvwriter = csv.writer(csvfile)
    # Write the fields and rows to the file
    #csvwriter.writerow(submission_fields)
    csvwriter.writerows(submissions_dataset)
    
with open(comments_filename, 'a', encoding="utf-8") as csvfile:
    # Create a writer object
    csvwriter = csv.writer(csvfile)
    # Write the fields and rows to the file
    #csvwriter.writerow(comment_fields)
    csvwriter.writerows(comments_dataset)


In [219]:
import pandas as pd
submissions_df = pd.read_csv(submissions_filename)

In [220]:
comments_df = pd.read_csv(comments_filename)

Note: # From qualitative analysis: a comment doesn't often have multiple coins+strong sentiment associated with each one, usually it's one coin mention and discussion, it may mention other coins but the comment itself will center around one of them and only have strong sentiment associated with that one coin. Comments like "BTC is amazing, Etherium sucks, Doge is the best" are rare. 

In [315]:
# Cleaning:
# Converted emojis to word representations, removed newlines and tabs
# \s on reddit means sarcasm - how to best represent this? Important for sentiment analysis

# remove removed comments
comments_df=comments_df[comments_df["Comment"]!="[removed]"]
# remove admin bot disclaimer comments 
comments_df = comments_df[~comments_df["Comment"].str.contains("\[pros\]\(/r/CryptoCurrency/")]

#comments_df["Comment"].str.split(expand=True).stack().value_counts()
#comments_df["Comment"].replace('\s', '*SARCASM*')
comments_df = comments_df.dropna(axis=0, how='any')

In [316]:
comments_df

,ID,Comment,Unixtime,Upvotes,SubmissionID
1,kwdqtko,Nobody knows shit,1.711309e+09,331,1bmshtm
2,kwdpxif,Ethereum could also top 20k. Or go back to 700...,1.711308e+09,549,1bmshtm
3,kwdsan3,Wtf is the point of following this sub every p...,1.711309e+09,137,1bmshtm
4,kwdqa7o,Could.,1.711309e+09,15,1bmshtm
5,kwdv5ch,It will either go up or down says my crystal b...,1.711310e+09,37,1bmshtm
...,...,...,...,...,...
411,kwfv14l,Joining the crypto gambleverse to dreaming to ...,1.711339e+09,1,1bmi3xr
412,kwg5z9w,"yes it is spread, welcome to the world of hidd...",1.711345e+09,1,1bmi3xr
413,kwbqla5,Ran away from Binance the moment they stopped ...,1.711280e+09,0,1bmi3xr
414,kwboc7t,I had a balance of 980€ in BTC after buying 10...,1.711278e+09,-6,1bmi3xr


In [ ]:
# TODO: BTC/Bitcoin mentions - need to map name/abbreviation to the same coin
# Can potentially be done by the LLM
# If not, can scrape the coinwire page and write a function to map

In [49]:
#TODO: load the base model using tensorflow

# Loading the base model, the tokenizer and the config
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [50]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [51]:
# Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, batched = True)

In [ ]:
#TODO: write a prompt

prompt = f'''For a given Reddit comment, extract the Cryptocurrency the comment mentions and the sentiment associated with that cryptocurrency. 
If there are multiple Cryptocurrencies mentioned in the comment, extract the main Cryptocurrency that the commenter is focusing on.
Classify the sentiment associated with this Cryptocurrency into positive, negative or neutral.

Comment: {}'''

In [332]:
comment_list = comments_df["Comment"]
example = comment_list[2]
example

'Ethereum could also top 20k. Or go back to 700$. Or 7k. Or anything else'

In [344]:
prompt = f"""
Give the sentiment of the following Reddit comment.

{example}
"""
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'], 
        )[0], 
        skip_special_tokens=True
)
    
print(f'INPUT PROMPT:\n{prompt}')
print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

C:\Users\olgakh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


INPUT PROMPT:

Give the sentiment of the following Reddit comment.

Ethereum could also top 20k. Or go back to 700$. Or 7k. Or anything else

MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
negative



In [340]:
inputs

{'input_ids': tensor([[ 6434,     8,  6493,    13,     8,   826,  1624,    26,   155,  1670,
             5, 28516,   228,    92,   420,   460,   157,     5,   955,   281,
           223,    12, 12283,  3229,     5,   955,   489,   157,     5,   955,
           959,  1307,  4892,  2998,   295,    10,     3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}